Coursera Capstone Project

In [75]:
import pandas as pd
import numpy as np

link = "https://www.zumper.com/rent-research/calgary-ab"

calgary_data = pd.read_html(link,header=0)[1]

calgary_data.loc[9, 'Neighborhood'] = 'Hillhurst'

In [76]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


for i in calgary_data.index:
    neigh = calgary_data.loc[i,'Neighborhood'] + ', Calgary, AB'
    geolocator = Nominatim(user_agent="ab_explorer")
    location = geolocator.geocode(neigh)
    calgary_data.at[i, 'Latitude'] = location.latitude
    calgary_data.at[i, 'Longitude'] = location.longitude

In [77]:
calgary_data

,Neighborhood,Median Rent,Latitude,Longitude
0,Beltline,"$1,213",51.040498,-114.072593
1,Downtown Calgary,"$1,208",51.046496,-114.050643
2,Mission,"$1,100",51.031758,-114.066720
3,Skyview Ranch,"$1,297",51.160534,-113.958135
4,Varsity,"$1,057",51.090998,-114.155759
5,Bridgeland,"$1,195",51.053801,-114.044310
6,The Properties,"$1,000",50.964185,-114.120937
7,Crescent Heights,"$1,088",51.059629,-114.060886
8,Bankview,$950,51.033887,-114.099518
9,Hillhurst,"$1,300",51.055414,-114.095564


In [78]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [79]:
address = 'Calgary, AB'

geolocator = Nominatim(user_agent="ab_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Calgary are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Calgary are 51.0534234, -114.0625892.


In [80]:
# create map of Toronto using latitude and longitude values
map_calgary = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(calgary_data['Latitude'], calgary_data['Longitude'], calgary_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_calgary)  
    
map_calgary

In [81]:
CLIENT_ID = 'XWWAH4HCU0AGKBED0I1LGPVE0LDQMFNJINLOTK1DCK5LDVGD' # your Foursquare ID
CLIENT_SECRET = 'PS5KMUMQ0VCNJ5T2KZF1NGT4CG1RRMQZOSEOSAJVK1J3Z3Q1' # your Foursquare Secret
ACCESS_TOKEN = 'SK2V2APCYIEVGYKOHCT5FGFE0EASNCM50CHXZP4QZLDG1DYM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XWWAH4HCU0AGKBED0I1LGPVE0LDQMFNJINLOTK1DCK5LDVGD
CLIENT_SECRET:PS5KMUMQ0VCNJ5T2KZF1NGT4CG1RRMQZOSEOSAJVK1J3Z3Q1


In [82]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [83]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
calgary_venues = getNearbyVenues(names=calgary_data['Neighborhood'],
                                   latitudes=calgary_data['Latitude'],
                                   longitudes=calgary_data['Longitude']
                                  )

Beltline
Downtown Calgary
Mission
Skyview Ranch
Varsity
Bridgeland
The Properties
Crescent Heights
Bankview
Hillhurst
Sunalta
Thorncliffe
Mount royal
Bowness
Kingsland
Capital Hill
Altadore
Acadia
Sunnyside
Banff Trail
Inglewood
Haysboro
Windsor Park
Forest Heights
West Hillhurst
South Calgary
Auburn Bay
Glenbrook
Palliser
Glenmorgan
Killarney
Rosscarrock
Radisson Heights
Mayland Heights
Applewood Park
Lakeview
Cedarbrae
University Heights
Quarry Park
Spruce Cliff
LIncoln Park
Wildwood


In [85]:
print(calgary_venues.shape)
calgary_venues.head()

(579, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beltline,51.040498,-114.072593,Milk Tiger Lounge,51.039357,-114.071403,Lounge
1,Beltline,51.040498,-114.072593,Central Memorial Park,51.041281,-114.070479,Park
2,Beltline,51.040498,-114.072593,Shelf Life Books,51.040737,-114.071526,Bookstore
3,Beltline,51.040498,-114.072593,Ox And Angela,51.037933,-114.073106,Latin American Restaurant
4,Beltline,51.040498,-114.072593,Yoga Passage,51.042799,-114.073123,Yoga Studio


In [86]:
calgary_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acadia,3,3,3,3,3,3
Altadore,4,4,4,4,4,4
Applewood Park,2,2,2,2,2,2
Auburn Bay,5,5,5,5,5,5
Banff Trail,19,19,19,19,19,19
Bankview,9,9,9,9,9,9
Beltline,54,54,54,54,54,54
Bowness,2,2,2,2,2,2
Bridgeland,21,21,21,21,21,21


In [87]:
print('There are {} uniques categories.'.format(len(calgary_venues['Venue Category'].unique())))

There are 141 uniques categories.


In [88]:
# one hot encoding
calgary_onehot = pd.get_dummies(calgary_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
calgary_onehot['Neighborhood'] = calgary_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [calgary_onehot.columns[-1]] + list(calgary_onehot.columns[:-1])
calgary_onehot = calgary_onehot[fixed_columns]

calgary_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Beltline,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Beltline,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Beltline,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Beltline,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Beltline,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [89]:
calgary_onehot.shape

(579, 142)

In [90]:
calgary_grouped = calgary_onehot.groupby('Neighborhood').mean().reset_index()
calgary_grouped

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Acadia,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Altadore,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Applewood Park,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Auburn Bay,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Banff Trail,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000
5,Bankview,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Beltline,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.037037,0.000000,0.018519,0.000000,0.055556
7,Bowness,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Bridgeland,0.000000,0.000000,0.000000,0.0,0.047619,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619
9,Capital Hill,0.000000,0.000000,0.000000,0.0,0.000000,0.100000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000


In [91]:
calgary_grouped.shape

(41, 142)

In [100]:
num_top_venues = 5

for hood in calgary_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = calgary_grouped[calgary_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acadia----
                        venue  freq
0                       Diner  0.33
1          Athletics & Sports  0.33
2  Construction & Landscaping  0.33
3                   Pet Store  0.00
4     New American Restaurant  0.00


----Altadore----
                   venue  freq
0       Greek Restaurant  0.25
1                Dog Run  0.25
2            Coffee Shop  0.25
3         Massage Studio  0.25
4  Performing Arts Venue  0.00


----Applewood Park----
                venue  freq
0                Park   0.5
1        Liquor Store   0.5
2  African Restaurant   0.0
3              Museum   0.0
4         Music Venue   0.0


----Auburn Bay----
               venue  freq
0  Other Repair Shop   0.2
1       Skating Rink   0.2
2               Lake   0.2
3       Tennis Court   0.2
4   Basketball Court   0.2


----Banff Trail----
                   venue  freq
0                  Hotel  0.32
1             Steakhouse  0.05
2            Pizza Place  0.05
3  Vietnamese Restaurant  0.05
4          

In [93]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [105]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = calgary_grouped['Neighborhood']

for ind in np.arange(calgary_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(calgary_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acadia,Diner,Athletics & Sports,Construction & Landscaping,Pet Store,New American Restaurant,Noodle House,Office,Other Repair Shop,Park,Performing Arts Venue
1,Altadore,Greek Restaurant,Dog Run,Coffee Shop,Massage Studio,Performing Arts Venue,New American Restaurant,Noodle House,Office,Other Repair Shop,Park
2,Applewood Park,Park,Liquor Store,African Restaurant,Museum,Music Venue,New American Restaurant,Noodle House,Office,Other Repair Shop,Performing Arts Venue
3,Auburn Bay,Other Repair Shop,Skating Rink,Lake,Tennis Court,Basketball Court,African Restaurant,Music Venue,New American Restaurant,Noodle House,Office
4,Banff Trail,Hotel,Steakhouse,Pizza Place,Vietnamese Restaurant,Coffee Shop,BBQ Joint,Seafood Restaurant,Inn,Liquor Store,Ice Cream Shop


In [106]:
# set number of clusters
kclusters = 5

calgary_grouped_clustering = calgary_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(calgary_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 0, 0, 0, 0, 4, 0, 0], dtype=int32)

In [107]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

calgary_merged = calgary_data.rename(columns={"Neighbourhood": "Neighborhood"})

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
calgary_merged = calgary_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

calgary_merged.dropna(axis=0, inplace=True)

calgary_merged['Cluster Labels'] = calgary_merged['Cluster Labels'].astype('int32')

calgary_merged.head() # check the last columns!

,Neighborhood,Median Rent,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beltline,"$1,213",51.040498,-114.072593,0.0,Pub,Yoga Studio,Restaurant,Bar,Cocktail Bar,French Restaurant,Gym / Fitness Center,Lounge,Pizza Place,Coffee Shop
1,Downtown Calgary,"$1,208",51.046496,-114.050643,0.0,Coffee Shop,Music Venue,Hotel,BBQ Joint,Bakery,Shopping Plaza,Museum,Hostel,Ice Cream Shop,History Museum
2,Mission,"$1,100",51.031758,-114.066720,0.0,Coffee Shop,French Restaurant,Japanese Restaurant,Indian Restaurant,Gym / Fitness Center,Cosmetics Shop,Bakery,Breakfast Spot,Burger Joint,Park
3,Skyview Ranch,"$1,297",51.160534,-113.958135,2.0,Home Service,African Restaurant,Moroccan Restaurant,Music Store,Music Venue,New American Restaurant,Noodle House,Office,Other Repair Shop,Park
4,Varsity,"$1,057",51.090998,-114.155759,0.0,Pizza Place,Gas Station,Yoga Studio,Café,Sandwich Place,Convenience Store,Shipping Store,Chinese Restaurant,Shopping Mall,Grocery Store


In [119]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(calgary_merged['Latitude'], calgary_merged['Longitude'], calgary_merged['Neighborhood'], calgary_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [168]:
calgary_merged

,Neighborhood,Median Rent,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beltline,"$1,213",51.040498,-114.072593,0,Pub,Yoga Studio,Restaurant,Bar,Cocktail Bar,French Restaurant,Gym / Fitness Center,Lounge,Pizza Place,Coffee Shop
1,Downtown Calgary,"$1,208",51.046496,-114.050643,0,Coffee Shop,Music Venue,Hotel,BBQ Joint,Bakery,Shopping Plaza,Museum,Hostel,Ice Cream Shop,History Museum
2,Mission,"$1,100",51.031758,-114.066720,0,Coffee Shop,French Restaurant,Japanese Restaurant,Indian Restaurant,Gym / Fitness Center,Cosmetics Shop,Bakery,Breakfast Spot,Burger Joint,Park
3,Skyview Ranch,"$1,297",51.160534,-113.958135,2,Home Service,African Restaurant,Moroccan Restaurant,Music Store,Music Venue,New American Restaurant,Noodle House,Office,Other Repair Shop,Park
4,Varsity,"$1,057",51.090998,-114.155759,0,Pizza Place,Gas Station,Yoga Studio,Café,Sandwich Place,Convenience Store,Shipping Store,Chinese Restaurant,Shopping Mall,Grocery Store
5,Bridgeland,"$1,195",51.053801,-114.044310,0,Italian Restaurant,Sushi Restaurant,Pizza Place,Burger Joint,Wine Shop,Sandwich Place,Pharmacy,Performing Arts Venue,Noodle House,Grocery Store
6,The Properties,"$1,000",50.964185,-114.120937,3,Liquor Store,Pizza Place,Fast Food Restaurant,Gas Station,Shopping Mall,Performing Arts Venue,Noodle House,Office,Other Repair Shop,Park
7,Crescent Heights,"$1,088",51.059629,-114.060886,0,Scenic Lookout,Restaurant,Mexican Restaurant,Falafel Restaurant,Poke Place,Diner,Sandwich Place,Seafood Restaurant,Steakhouse,Sushi Restaurant
8,Bankview,$950,51.033887,-114.099518,0,Pizza Place,Spa,Convenience Store,Health & Beauty Service,Candy Store,Tattoo Parlor,Coffee Shop,Grocery Store,Pharmacy,Pet Store
9,Hillhurst,"$1,300",51.055414,-114.095564,0,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Chinese Restaurant,Wine Shop,Sandwich Place,Restaurant,Pub,Pharmacy,Moroccan Restaurant
